#  Ambisonics Upscaler : FOA → HOA2


## 1. Setup & Installation

In [1]:
!pip install -q torch torchaudio datasets huggingface_hub tensorboard tqdm soundfile scipy

In [2]:
import os
import io
import gc
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchaudio
from datasets import load_dataset
from huggingface_hub import HfApi, login, upload_folder
from tqdm.auto import tqdm
from datetime import datetime
from pathlib import Path
import numpy as np
import soundfile as sf
import random
import math

def flush_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
    print("✅ GPU Memory flushed")

def check_memory():
    if not torch.cuda.is_available():
        print("No CUDA available")
        return

    t = torch.cuda.get_device_properties(0).total_memory / 1024**3
    r = torch.cuda.memory_reserved(0) / 1024**3
    a = torch.cuda.memory_allocated(0) / 1024**3

    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Total: {t:.2f} GB")
    print(f"Reserved: {r:.2f} GB")
    print(f"Allocated: {a:.2f} GB")
    print(f"Free: {t-a:.2f} GB")

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.9.0+cu126
CUDA available: True


## 2. HuggingFace Auth

In [ ]:
HF_TOKEN = ""
HF_DATASET_REPO = "jack635/ambisonic-dataset-final"
HF_OUTPUT_REPO = "jack635/ambisonic-dataset-final"
login(token=HF_TOKEN)
api = HfApi()

## 3. Configuration FIXED

In [4]:
CONFIG = {
    "foa_channels": 4,
    "hoa2_channels": 9,
    "sample_rate": 48000,
    "segment_length": 48000,

    # ARCHITECTURE
    "encoder_dim": 512,
    "hidden_dim": 128,
    "kernel_size": 3,
    "num_layers": 7,
    "num_repeats": 2,

    # TRAINING STABLE - CORRECTIONS APPLIQUÉES
    "batch_size": 4,
    "learning_rate": 1e-4,  # ✅ RÉDUIT de 5e-4 à 1e-4
    "weight_decay": 1e-4,
    "num_epochs": 400,
    "grad_clip": 5.0,  # ✅ AUGMENTÉ de 1.0 à 5.0

    # WARMUP
    "warmup_epochs": 10,  # ✅ AJOUTÉ
    "warmup_lr_start": 1e-5,  # ✅ AJOUTÉ

    # EARLY STOPPING
    "early_stopping_patience": 40,

    # LOSS STABLE
    "sc_weight": 1.0,
    "mag_weight": 1.0,
    "time_weight": 0.2,

    # AUGMENTATION
    "aug_prob": 0.3,
    "gain_range": [-12, 3],
    "noise_snr": [35, 65],
    "rotation_prob": 0.4,

    "save_every": 10,
    "upload_every": 20,
    "generate_samples": 5,
}

OUTPUT_DIR = Path("/content/ambi_upscaler_robust_fixed")
for d in ["checkpoints", "samples", "logs"]:
    (OUTPUT_DIR / d).mkdir(parents=True, exist_ok=True)

print("✅ Configuration loaded")
print(f"  LR: {CONFIG['learning_rate']} (warmup from {CONFIG['warmup_lr_start']})")
print(f"  Grad Clip: {CONFIG['grad_clip']}")
print(f"  Warmup: {CONFIG['warmup_epochs']} epochs")

✅ Configuration loaded
  LR: 0.0001 (warmup from 1e-05)
  Grad Clip: 5.0
  Warmup: 10 epochs


## 4. Ambisonics Rotation

In [5]:
def rotate_ambisonics_azimuth(audio: torch.Tensor, angle_rad: float) -> torch.Tensor:
    n_channels = audio.shape[0]
    rotated = audio.clone()

    cos_a = math.cos(angle_rad)
    sin_a = math.sin(angle_rad)
    cos_2a = math.cos(2 * angle_rad)
    sin_2a = math.sin(2 * angle_rad)

    # Order 1: Y, Z, X
    if n_channels >= 4:
        Y, Z, X = audio[1], audio[2], audio[3]
        rotated[1] = Y * cos_a + X * sin_a
        rotated[3] = -Y * sin_a + X * cos_a

    # Order 2: V, T, R, S, U
    if n_channels >= 9:
        V, T, R, S, U = audio[4], audio[5], audio[6], audio[7], audio[8]
        rotated[4] = V * cos_2a + U * sin_2a
        rotated[5] = T * cos_a + S * sin_a
        rotated[7] = -T * sin_a + S * cos_a
        rotated[8] = -V * sin_2a + U * cos_2a

    rotated = torch.nan_to_num(rotated, 0.0)
    return rotated

## 5. Robust Dataset Loader

In [6]:
class HFAmbisonicsDatasetRobust(Dataset):
    def __init__(self, hf_dataset, config, split="train"):
        raw_data = hf_dataset[split] if split in hf_dataset else hf_dataset
        self.valid_indices = []
        for i in range(len(raw_data)):
            item = raw_data[i]
            if not item.get('is_impulse_response', False) and item.get('channels', 0) >= 9:
                self.valid_indices.append(i)

        self.data = raw_data
        self.config = config
        self.segment_length = config["segment_length"]
        self.target_sr = config["sample_rate"]
        self.training = (split == "train")
        print(f"✅ Accepted {len(self.valid_indices)} files for {split}")

    def __len__(self):
        return len(self.valid_indices)

    def _load_audio(self, audio_data):
        if isinstance(audio_data, dict):
            if 'array' in audio_data:
                return torch.tensor(audio_data['array'], dtype=torch.float32), audio_data.get('sampling_rate', self.target_sr)
            if 'bytes' in audio_data:
                audio, sr = sf.read(io.BytesIO(audio_data['bytes']))
                return torch.tensor(audio, dtype=torch.float32).T, sr
        return torch.tensor(audio_data, dtype=torch.float32), self.target_sr

    def _apply_augmentation(self, audio):
        if not self.training:
            return audio

        audio = torch.nan_to_num(audio, 0.0)

        # 1. Rotation
        if random.random() < self.config.get("rotation_prob", 0.0):
            angle = random.uniform(0, 2 * math.pi)
            audio = rotate_ambisonics_azimuth(audio, angle)

        if random.random() < self.config["aug_prob"]:
            # 2. Gain
            gain_db = random.uniform(self.config["gain_range"][0], self.config["gain_range"][1])
            audio = audio * (10 ** (gain_db / 20))

            # 3. Noise
            snr_db = random.uniform(self.config["noise_snr"][0], self.config["noise_snr"][1])
            sig_pow = audio.pow(2).mean() + 1e-10
            noise_pow = sig_pow / (10 ** (snr_db / 10))
            audio = audio + torch.randn_like(audio) * torch.sqrt(noise_pow)

        audio = torch.nan_to_num(audio, 0.0)
        audio = torch.clamp(audio, -1.0, 1.0)
        return audio

    def __getitem__(self, idx):
        real_idx = self.valid_indices[idx]
        item = self.data[real_idx]
        audio, sr = self._load_audio(item['audio'])

        if sr != self.target_sr:
            audio = torchaudio.functional.resample(audio, int(sr), self.target_sr)

        if audio.dim() == 1:
            audio = audio.unsqueeze(0)
        elif audio.shape[0] > audio.shape[1]:
            audio = audio.T

        if audio.shape[1] < self.segment_length:
            audio = F.pad(audio, (0, self.segment_length - audio.shape[1]))
        else:
            start = random.randint(0, audio.shape[1] - self.segment_length)
            audio = audio[:, start:start + self.segment_length]

        audio = self._apply_augmentation(audio)
        return audio[:4], audio[:9]

print("Loading dataset...")
hf_dataset = load_dataset(HF_DATASET_REPO, token=HF_TOKEN)
from datasets import Audio
hf_dataset = hf_dataset.cast_column("audio", Audio(decode=False))

full_ds = HFAmbisonicsDatasetRobust(hf_dataset, CONFIG, split="train")
train_size = int(0.9 * len(full_ds))
train_ds, val_ds = torch.utils.data.random_split(full_ds, [train_size, len(full_ds)-train_size])
train_loader = DataLoader(train_ds, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=CONFIG["batch_size"], shuffle=False)

print("\n🔍 Audio Stats:")
foa, hoa2 = next(iter(train_loader))
print(f"FOA: min={foa.min():.4f}, max={foa.max():.4f}, std={foa.std():.4f}")
print(f"HOA2: min={hoa2.min():.4f}, max={hoa2.max():.4f}, std={hoa2.std():.4f}")

Loading dataset...


README.md:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/25 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/25 [00:00<?, ?it/s]

data/train-00000-of-00025.parquet:   0%|          | 0.00/277M [00:00<?, ?B/s]

...

data/train-00023-of-00025.parquet:   0%|          | 0.00/927M [00:00<?, ?B/s]

data/train-00024-of-00025.parquet:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/282 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

✅ Accepted 282 files for train

🔍 Audio Stats:
FOA: min=-0.4112, max=0.4441, std=0.0351
HOA2: min=-0.5404, max=0.5380, std=0.0414


## 6. Model Architecture & Stable Loss

In [7]:
class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation=1):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, padding=self.padding, dilation=dilation)

    def forward(self, x):
        x = self.conv(x)
        return x[:, :, :-self.padding] if self.padding > 0 else x

class GlobalLayerNorm(nn.Module):
    def __init__(self, dim, eps=1e-5):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(dim))
        self.beta = nn.Parameter(torch.zeros(dim))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(dim=(1, 2), keepdim=True)
        var = x.var(dim=(1, 2), keepdim=True, unbiased=False)
        return (x - mean) / torch.sqrt(var + self.eps) * self.gamma.view(1, -1, 1) + self.beta.view(1, -1, 1)

class CausalConvBlock(nn.Module):
    def __init__(self, hidden_dim, kernel_size, dilation):
        super().__init__()
        self.conv1 = CausalConv1d(hidden_dim, hidden_dim, kernel_size, dilation)
        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim, 1)
        self.norm1 = GlobalLayerNorm(hidden_dim)
        self.norm2 = GlobalLayerNorm(hidden_dim)
        self.prelu = nn.PReLU()

    def forward(self, x):
        residual = x
        h = self.prelu(self.norm1(self.conv1(x)))
        return residual + self.norm2(self.conv2(h))

class AmbiUpscalerHOA2(nn.Module):
    def __init__(self, config):
        super().__init__()
        d = config["encoder_dim"]
        k = config["kernel_size"]
        self.encoder = nn.Sequential(
            nn.Conv1d(4, d, 1),
            nn.PReLU(),
            GlobalLayerNorm(d)
        )
        self.blocks = nn.ModuleList([
            CausalConvBlock(d, k, 2**(i % config["num_layers"]))
            for i in range(config["num_layers"] * config["num_repeats"])
        ])
        self.decoder = nn.Sequential(
            nn.Conv1d(d, d, 1),
            nn.PReLU(),
            GlobalLayerNorm(d),
            nn.Conv1d(d, 9, 1)
        )

    def forward(self, x):
        h = self.encoder(x)
        for block in self.blocks:
            h = block(h)
        return self.decoder(h)

class HiFiLossStable(nn.Module):
    """✅ Loss stable avec epsilons corrigés"""
    def __init__(self, config):
        super().__init__()
        self.sc_w = config["sc_weight"]
        self.mag_w = config["mag_weight"]
        self.time_w = config["time_weight"]

    def forward(self, pred, target):
        # Check NaN/Inf
        if torch.isnan(pred).any() or torch.isinf(pred).any():
            pred = torch.nan_to_num(pred, 0.0)
        if torch.isnan(target).any() or torch.isinf(target).any():
            target = torch.nan_to_num(target, 0.0)

        time_loss = F.l1_loss(pred, target)

        window = torch.hann_window(1024, device=pred.device)
        stft_p = torch.stft(pred.reshape(-1, pred.shape[-1]), 1024, 256, window=window, return_complex=True).abs()
        stft_t = torch.stft(target.reshape(-1, target.shape[-1]), 1024, 256, window=window, return_complex=True).abs()

        # ✅ EPSILON AUGMENTÉ pour stabilité (1e-3 au lieu de 1e-8)
        sc_loss = torch.norm(stft_t - stft_p, p="fro") / (torch.norm(stft_t, p="fro") + 1e-3)

        # ✅ CLAMP avant log pour éviter les explosions
        stft_p_safe = torch.clamp(stft_p, min=1e-5)
        stft_t_safe = torch.clamp(stft_t, min=1e-5)
        mag_loss = F.l1_loss(torch.log(stft_p_safe + 1e-5), torch.log(stft_t_safe + 1e-5))

        return self.sc_w * sc_loss + self.mag_w * mag_loss + self.time_w * time_loss

print("✅ Model & Loss defined")

✅ Model & Loss defined


## 7. Training avec Warmup & Monitoring

In [8]:
flush_memory()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AmbiUpscalerHOA2(CONFIG).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG["warmup_lr_start"], weight_decay=CONFIG["weight_decay"])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG["num_epochs"] - CONFIG["warmup_epochs"])
writer = SummaryWriter(log_dir=str(OUTPUT_DIR / "logs"))
loss_fn = HiFiLossStable(CONFIG)

print("\n🚀 Starting training...")
print(f"Device: {device}")
print(f"Model params: {sum(p.numel() for p in model.parameters()):,}")

best_loss = float('inf')
patience_counter = 0

for epoch in range(CONFIG["num_epochs"]):
    # ✅ WARMUP Learning Rate
    if epoch < CONFIG["warmup_epochs"]:
        lr = CONFIG["warmup_lr_start"] + (CONFIG["learning_rate"] - CONFIG["warmup_lr_start"]) * epoch / CONFIG["warmup_epochs"]
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

    # Training
    model.train()
    total_train_loss = 0
    grad_norms = []

    pbar = tqdm(train_loader, desc=f"Epoch {epoch}")
    for foa, hoa2 in pbar:
        foa, hoa2 = foa.to(device), hoa2.to(device)

        optimizer.zero_grad()
        pred = model(foa)
        loss = loss_fn(pred, hoa2)

        # ✅ SKIP NaN/Inf losses
        if torch.isnan(loss) or torch.isinf(loss):
            print(f"⚠️ Skipped NaN/Inf loss at epoch {epoch}")
            continue

        loss.backward()
        gnorm = torch.nn.utils.clip_grad_norm_(model.parameters(), CONFIG["grad_clip"])
        grad_norms.append(gnorm.item())
        optimizer.step()

        total_train_loss += loss.item()

        # Live monitoring
        mem = torch.cuda.memory_allocated(0) / 1024**3 if torch.cuda.is_available() else 0
        pbar.set_postfix({
            "loss": f"{loss.item():.3f}",
            "gnorm": f"{gnorm:.1f}",
            "mem": f"{mem:.1f}GB",
            "lr": f"{optimizer.param_groups[0]['lr']:.2e}"
        })

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for foa, hoa2 in val_loader:
            loss = loss_fn(model(foa.to(device)), hoa2.to(device))
            if not (torch.isnan(loss) or torch.isinf(loss)):
                val_loss += loss.item()

    val_loss /= len(val_loader)
    avg_train_loss = total_train_loss / len(train_loader)
    avg_gnorm = sum(grad_norms) / len(grad_norms) if grad_norms else 0

    # Logging
    writer.add_scalar("Loss/Train", avg_train_loss, epoch)
    writer.add_scalar("Loss/Val", val_loss, epoch)
    writer.add_scalar("Grad/Norm", avg_gnorm, epoch)
    writer.add_scalar("LR", optimizer.param_groups[0]['lr'], epoch)

    print(f"Epoch {epoch}: Train={avg_train_loss:.4f}, Val={val_loss:.4f}, Grad={avg_gnorm:.2f}, LR={optimizer.param_groups[0]['lr']:.2e}")

    # ✅ WARNING pour gradients élevés
    if avg_gnorm > 100:
        print(f"⚠️ High gradient norm: {avg_gnorm:.0f}")

    # Save best model
    if val_loss < best_loss:
        best_loss = val_loss
        patience_counter = 0
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_loss,
        }, OUTPUT_DIR / "checkpoints/best_robust.pt")
        print(f"✅ New best model saved (Val Loss: {val_loss:.4f})")
    else:
        patience_counter += 1

    # Early stopping
    if patience_counter >= CONFIG["early_stopping_patience"]:
        print(f"🛑 Early stopping at epoch {epoch}")
        break

    # Step scheduler (after warmup)
    if epoch >= CONFIG["warmup_epochs"]:
        scheduler.step()

writer.close()
print("\n✅ Training Complete!")
check_memory()

✅ GPU Memory flushed

🚀 Starting training...
Device: cuda
Model params: 14,994,969


Epoch 0:   0%|          | 0/64 [00:00<?, ?it/s]

Epoch 0: Train=4.9090, Val=2.4811, Grad=890.28, LR=1.00e-05
⚠️ High gradient norm: 890
✅ New best model saved (Val Loss: 2.4811)


Epoch 1:   0%|          | 0/64 [00:00<?, ?it/s]

Epoch 1: Train=2.7330, Val=3.7204, Grad=575.07, LR=1.90e-05
⚠️ High gradient norm: 575


...

Epoch 140: Train=1.3057, Val=0.9951, Grad=326.33, LR=6.83e-05
⚠️ High gradient norm: 326


Epoch 141:   0%|          | 0/64 [00:00<?, ?it/s]

Epoch 141: Train=1.1903, Val=1.0890, Grad=261.56, LR=6.79e-05
⚠️ High gradient norm: 262


Epoch 142:   0%|          | 0/64 [00:00<?, ?it/s]

Epoch 142: Train=1.2367, Val=1.0056, Grad=299.12, LR=6.76e-05
⚠️ High gradient norm: 299
🛑 Early stopping at epoch 142

✅ Training Complete!
GPU: NVIDIA A100-SXM4-80GB
Total: 79.32 GB
Reserved: 53.07 GB
Allocated: 0.24 GB
Free: 79.08 GB


## 8. Load Best Model & Test

In [14]:
from google.colab import files

files.download("/content/ambi_upscaler_robust_fixed/checkpoints/best_robust.pt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>